In [1]:
using Knet, Plots, NBInclude;
nbinclude("deepppiutils.ipynb");

In [42]:
function loaddata(pairsfilepath; Atype=gpu() >= 0 ? KnetArray{Float32} : Array{Float32})
    pairsfile = open(pairsfilepath)
    lines = readlines(pairsfile)
    close(pairsfile)

    samples = lines[4:end];
    concatAB1 = []
    ygold = Array{UInt8,1}(length(samples));
    i = 1;
    for s in samples
        s = split(s, "\t");
        ygold[i] = parse(UInt8, s[7]) + 0x01;
        s = parse.(Float32, s[3:end-1]);
        push!(concatAB1, reshape(s, 1, 4))
        i += 1;
    end
    concatAB = vcat(map(Atype, concatAB1)...)
    return concatAB, ygold
end;

In [43]:
function trn_tst_split(data, ygold, trnper, tstper; batchsize=64)
    nosamples = size(data,1)
    notrn = Int(floor(trnper * nosamples))
    notst = nosamples - notrn
    ind = randperm(nosamples)
    
    xtrn = data[ind[1:notrn],:];
    ytrn = ygold[ind[1:notrn]];
    
    xtst = data[ind[notrn+1:notrn+notst], :];
    ytst = ygold[ind[notrn+1:notrn+notst]];
    
    dtrn = minibatchi(xtrn',ytrn,batchsize);
    dtst = minibatchi(xtst',ytst,batchsize);
    
    return dtrn, dtst
end;

In [44]:
humangoldfile = "test_datasets/Human/human_gold.tab";
humansilverfile = "test_datasets/Human/human_silver.tab";
humanallfile = "test_datasets/Human/human_all.tab";

# training sets
humangold_dataset,humangold_label=loaddata(humangoldfile);
humansilver_dataset,humansilver_label=loaddata(humansilverfile);

# test set
humanall_dataset,humanall_label=loaddata(humanallfile);

In [ ]:
# the number of hidden units in the hidden layers of the DeepPPI-CON model
## override number of inputs from deeputils
NOINPUTS = 4
HIDDENSSEP = Any[NOINPUTS, 512, 256, 128]; 
HIDDENSMER = Any[256, 128, NOOUTPUTS]
NOEPOCH = 100;
BATCHSIZE = 32;
PDROP = (0, 0.2);
accuracy_ =[]
recall=[]
specifity=[]
precision= []
mcc=[]
f1=[]
npv=[]
accuracyt= recalli=specifityi=precisioni=mcci = 0.0
for i in 1:5
    #setseed(i);
    #wa = winit(HIDDENSSEP...);
    #wb = winit(HIDDENSSEP...);
    #wMerged = winit(HIDDENSMER...);
    #w = vcat(wa, wb, wMerged);
    w = deepcopy(wl)
    
    #dtrn, ddev, dtst = dividedataset(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE);
    dtrn, dtst = trn_tst_split(yeast_dataset, yeast_label, trnper, tstper; batchsize= BATCHSIZE);
    
    optims = params(w; optim="Momentum", lr=0.01, gamma=0.9);
    #@time trnloss, trnerr, tstloss, tsterr=trainSep!(w, optims, dtrn, predictSep, ddev; pdrop=PDROP, epochs=NOEPOCH) 
    @time trainSep!(w, optims, dtrn, predictSep, dtst; pdrop=PDROP, epochs=NOEPOCH) 
    
    println("Yeast Dataset, fold: ", i)
    println("Training: min. loss =",loss(w,dtrn,predictSep),", min. error =",zeroone(w,dtrn,predictSep))  
    println("Test: min. loss =",loss(w,dtst,predictSep),", min. error =",zeroone(w,dtst,predictSep))  
    
    accuracyt,recalli,specifityi,precisioni,mcci,f1i,npvi = modelevaluation(w, dtst, predictSep; p=true);
    push!(accuracy_, accuracyt)
    push!(recall, recalli)
    push!(specifity, specifityi)
    push!(precision, precisioni)
    push!(mcc, mcci)
    push!(f1, f1i)
    push!(npv, npvi)
    
end